# 🤖 Détection de SMS Spam avec BERT (Fine-tuning complet)

Projet NLP utilisant le dataset [UCI SMS Spam](https://huggingface.co/datasets/ucirvine/sms_spam) et un modèle BERT pré-entraîné pour classifier des messages SMS comme spam ou ham (non-spam).

In [ ]:

# 📦 Installation des librairies (versions compatibles)
!pip install -q transformers==4.3.3 datasets evaluate


## 📥 Chargement et exploration du dataset

In [ ]:

from datasets import load_dataset
import pandas as pd

# Chargement du dataset SMS Spam
raw = load_dataset("ucirvine/sms_spam")
raw_df = pd.DataFrame(raw['train'])

# Aperçu
print("Taille totale :", len(raw_df))
raw_df.head()


In [ ]:

# Séparation en train/validation
train_df = raw_df.sample(frac=0.8, random_state=42)
val_df = raw_df.drop(train_df.index)

from datasets import Dataset
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds = Dataset.from_pandas(val_df.reset_index(drop=True))

train_ds.features


## ✍️ Tokenisation avec BERT

In [ ]:

from transformers import BertTokenizer

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

def tokenize_fn(examples):
    return tokenizer(
        examples["sms"],
        padding="max_length",
        truncation=True,
        max_length=64
    )

train_tok = train_ds.map(tokenize_fn, batched=True)
val_tok = val_ds.map(tokenize_fn, batched=True)


## 🧠 Chargement du modèle BERT pour classification binaire

In [ ]:

import torch
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)


## 📏 Définition des métriques (Accuracy, Precision, Recall, F1)

In [ ]:

import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy":  accuracy.compute(predictions=preds, references=labels)["accuracy"], 
        "precision": precision.compute(predictions=preds, references=labels)["precision"],
        "recall":    recall.compute(predictions=preds, references=labels)["recall"],
        "f1":        f1.compute(predictions=preds, references=labels)["f1"]
    }


## ⚙️ Configuration de l'entraînement

In [ ]:

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-spam-output",
    do_train=True,
    do_eval=True,
    eval_steps=500,
    save_steps=500,
    logging_dir="./logs",
    logging_steps=500,

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to=None,
    save_total_limit=1,
)


## 🚀 Entraînement du modèle BERT

In [ ]:

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    compute_metrics=compute_metrics,
)

trainer.train()


## 📊 Évaluation finale

In [ ]:

metrics = trainer.evaluate()
print(metrics)


## 🧠 Lecture métier

- **Accuracy** : part globale de bonnes prédictions
- **Recall** : capacité à repérer les *spams* (important pour ne rien laisser passer)
- **Precision** : capacité à ne pas classer à tort un *ham* comme spam

🔍 Un bon modèle aura un **F1-score élevé** : bon équilibre entre rappel et précision.

## 🧩 Visualisation : Matrice de confusion

In [ ]:

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Prédictions sur l'ensemble de validation
predictions = trainer.predict(val_tok)
y_pred = predictions.predictions.argmax(axis=-1)
y_true = predictions.label_ids

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Ham", "Spam"])

plt.figure(figsize=(6,6))
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title("Matrice de confusion - Validation")
plt.show()


## 🚀 Intégration (facultative) avec Weights & Biases

In [ ]:

# Optionnel : activer Weights & Biases (nécessite un compte)
# pip install wandb
# import wandb
# wandb.login()

# Ensuite, dans TrainingArguments :
# report_to="wandb"
